In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

def load_images(folder_path, label):
    images = []
    labels = []
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (64, 64))
        img = img / 255.0
        images.append(img)
        labels.append(label)
    return images, labels

open_eye_imgs, open_labels = load_images('D:\\Gen_AI\\DRIVER DROSWSINESS\\ImprovementSet\\opened', 0)
closed_eye_imgs, closed_labels = load_images('D:\\Gen_AI\\DRIVER DROSWSINESS\\ImprovementSet\\closed', 1)

X = np.array(open_eye_imgs + closed_eye_imgs).reshape(-1, 64, 64, 1)
y = np.array(open_labels + closed_labels)

In [3]:

# Split into train and temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Split temp into validation and test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Training set: {X_train.shape}, {y_train.shape}")
print(f"Validation set: {X_val.shape}, {y_val.shape}")
print(f"Test set: {X_test.shape}, {y_test.shape}")


Training set: (3282, 64, 64, 1), (3282,)
Validation set: (410, 64, 64, 1), (410,)
Test set: (411, 64, 64, 1), (411,)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, y_val)
)


In [15]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.9538 - loss: 0.1140
Test Accuracy: 95.38%


In [17]:
model.save("driver_drowsiness_model.h5")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import pygame

# Initialize pygame for audio
pygame.mixer.init()
alert_sound = pygame.mixer.Sound("Open CV/alert.wav")

# Load your model
model = load_model('Open CV/driver_drowsiness_model.h5')

# Load Haar cascade for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def preprocess_eye(eye_img):
    # Resize to 64x64 as per your model requirements
    eye_img = cv2.resize(eye_img, (64, 64))
    eye_img = eye_img.astype('float32') / 255.0
    
    # Add channel dimension for grayscale
    if len(eye_img.shape) == 2:
        eye_img = np.expand_dims(eye_img, axis=-1)
    
    eye_img = np.expand_dims(eye_img, axis=0)
    return eye_img

def main():
    cap = cv2.VideoCapture(0)
    closed_counter = 0
    alarm_active = False
    
    # Set camera resolution
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    print("Starting 64x64 model detection...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Detect eyes
        eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
        
        eyes_closed = 0
        total_eyes = len(eyes)
        
        for (x, y, w, h) in eyes:
            # Add padding to eye region
            padding = 5
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(frame.shape[1] - x, w + 2 * padding)
            h = min(frame.shape[0] - y, h + 2 * padding)
            
            eye_region = gray[y:y+h, x:x+w]
            
            if eye_region.size > 0:
                try:
                    # Preprocess for your 64x64 model
                    processed_eye = preprocess_eye(eye_region)
                    
                    # Predict
                    prediction = model.predict(processed_eye, verbose=0)
                    
                    # Adjust based on your model output
                    if prediction.shape[1] == 1:  # Binary classification
                        is_closed = prediction[0][0] > 0.5
                        confidence = prediction[0][0] if is_closed else 1 - prediction[0][0]
                    else:  # Multi-class
                        is_closed = np.argmax(prediction) == 0
                        confidence = np.max(prediction)
                    
                    # Draw rectangle and label
                    color = (0, 0, 255) if is_closed else (0, 255, 0)
                    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                    cv2.putText(frame, f"{'Closed' if is_closed else 'Open'} ({confidence:.2f})", 
                               (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                    if is_closed:
                        eyes_closed += 1
                        
                except Exception as e:
                    print(f"Error processing eye: {e}")
                    continue
        
        # Drowsiness logic
        if total_eyes > 0:
            if eyes_closed == total_eyes:  # All detected eyes are closed
                closed_counter += 1
                if closed_counter >= 10:  # Adjust threshold as needed
                    if not alarm_active:
                        alert_sound.play()
                        alarm_active = True
                    cv2.putText(frame, "DROWSY! WAKE UP!", (50, 50), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                    status_text = "ALERT: DROWSINESS DETECTED!"
                else:
                    status_text = f"Warning: Eyes closing... ({closed_counter}/10)"
            else:
                closed_counter = max(0, closed_counter - 1)
                alarm_active = False
                status_text = "Normal: Eyes Open"
        else:
            status_text = "No eyes detected"
            closed_counter = 0
            alarm_active = False
        
        # Display status
        cv2.putText(frame, status_text, (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Drowsiness Detection - 64x64 Model', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.10.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


Starting 64x64 model detection...
